# Tutorial

This tutorial demonstrates the main workflow for using `mpllayout`.

The workflow involves just a few high-level steps:
1. Create a layout object to store the layout, `layout = Layout()`
2. Add geometric primitives to `layout` using `layout.add_prim`. These primitives represent figure elements.
3. Add geometric constraints to `layout` using `layout.add_constraint` to constrain the primitives.
4. Solve the constrained layout of primitives using `constrained_prims = solve(layout.root_prim, *layout.flat_constraints())`
5. Generate a figure and axes to plot in using `fig, axs = subplots(constrained_prims)`

The generated `fig` and `axs` will reflect the constrained layout.

In [ ]:
import numpy as np

import matplotlib as mpl
from matplotlib import pyplot as plt

# `layout` contains the `Layout` class and related functions
from mpllayout import layout as lay
# `geometry` contains primitive and constraint objects
from mpllayout import geometry as geo
# `solve` is used to solve the constrained layout
from mpllayout.solver import solve

# `subplots` and `update_subplots` are used to create matplotlib figure and
# axes objects from geometric primitives
from mpllayout.matplotlibutils import subplots, update_subplots

# `ui` contains functions to visualize primitives
from mpllayout import ui

## 1. Create the layout

In [ ]:
# Create the layout to store constraints and primitives
layout = lay.Layout()

## 2. Add geometric primitives

Geometric primitives represent geometry and are defined in `mpllayout.primitives`.
Each primitive consists of a parameter vector (`primitive.value`) with child primitives (`primitive["child_key"]`).
For example:

* `Point` represents a point and has a parameter vector containing its coordinates with no child primitives
* `Line` represents a straight line segment, has no parameter vector, and contains two points representing the start point (`line["Point0"]`) and end point (`line["Point1"]`)
* Other primitives are documented in the module

Geometric primitives are added using the signature
`layout.add_prim(primitive, key)`
where `primitive` is a geometric primitive object and `key` is a string used to identify it.

In [ ]:
# A `Quadrilateral` is a 4 sided polygon which can be used to represent the figure box.
# Naming the quad "Figure" will cause the `subplots` command to create a figure of the same size.
layout.add_prim(geo.Quadrilateral(), "Figure")

# The `Axes` primitive is a collection of quadrilaterals and points used to represent an axes.
# The child primitives of `Axes` are
# - "Frame": a `Quadrilateral` representing the plotting area of the axes
# - "XAxis": a `Quadrilateral` bounding x-axis ticks and tick labels
# - "XAxisLabel": a `Point` for the x-axis label text anchor
# - "YAxis": a `Quadrilateral` bounding y-axis ticks and tick labels
# - "YAxisLabel": a `Point` for the y-axis label text anchor
# The x/y axis can be optionally included by kwargs as seen below
layout.add_prim(geo.Axes(xaxis=True, yaxis=True), "Axes")

In [ ]:

## Make all quadrilaterals rectangular/boxes
# NOTE: This step is needed because `Quadrilateral` by default don't have
# to be rectangles
layout.add_constraint(geo.Box(), ("Figure",), ())
layout.add_constraint(geo.Box(), ("Axes/Frame",), ())
layout.add_constraint(geo.Box(), ("Axes/XAxis",), ())
layout.add_constraint(geo.Box(), ("Axes/YAxis",), ())

## Constrain the figure size + position
# Fix the figure bottom left to the origin
layout.add_constraint(geo.Fix(), ("Figure/Line0/Point0",), (np.array([0, 0]),))

# Figure the figure width and height
fig_width, fig_height = 6, 3
layout.add_constraint(geo.XLength(), ("Figure/Line0",), (fig_width,))
layout.add_constraint(geo.YLength(), ("Figure/Line1",), (fig_height,))

## Constrain margins around the axes to the figure
# Constrain left/right margins
margin_left = 0.1
margin_right = 1/4

layout.add_constraint(
    geo.InnerMargin(side='left'), ("Axes/Frame", "Figure"), (margin_left,)
)
layout.add_constraint(
    geo.InnerMargin(side='right'), ("Axes/YAxis", "Figure"), (margin_right,)
)

# Constrain top/bottom margins
margin_top = 1/4
margin_bottom = 0.1
layout.add_constraint(
    geo.InnerMargin(side='bottom'), ("Axes/Frame", "Figure"), (margin_bottom,)
)
layout.add_constraint(
    geo.InnerMargin(side='top'), ("Axes/XAxis", "Figure"), (margin_top,)
)

## Position the x axis on top and the y axis on the bottom
# When creating axes from the primitives, `lplt.subplots` will detect axis
# positions and set axis properties to reflect them.
layout.add_constraint(geo.PositionXAxis(bottom=False, top=True), ("Axes", ), ())
layout.add_constraint(geo.PositionYAxis(left=False, right=True), ("Axes", ), ())

# Link x/y axis width/height to axis sizes in matplotlib.
# Axis sizes change depending on the size of their tick labels so the
# axis width/height must be linked to matplotlib and updated from plot
# elements.
layout.add_constraint(
    geo.XAxisHeight(), ("Axes/XAxis",), (None,),
)
layout.add_constraint(
    geo.YAxisWidth(), ("Axes/YAxis",), (None,),
)

## Position the x/y axis label text anchors
# When creating axes from the primitives, `lplt.subplots` will detect these
# and set their locations
on_line = geo.RelativePointOnLineDistance()
to_line = geo.PointToLineDistance()

## Pad x/y axis label from the axis bbox
pad = 1/16
layout.add_constraint(to_line, ("Axes/XAxisLabel", "Axes/XAxis/Line2"), {"distance": pad, "reverse": True})
layout.add_constraint(to_line, ("Axes/YAxisLabel", "Axes/YAxis/Line1"), {"distance": pad, "reverse": True})

## Center the axis labels halfway along the axes width/height
layout.add_constraint(geo.PositionXAxisLabel(), ("Axes",), {"distance": 0.5})
layout.add_constraint(geo.PositionYAxisLabel(), ("Axes",), {"distance": 0.5})